In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D
from city_sustainability.quality import life_quality
from city_sustainability.models import unet_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from city_sustainability.preprocessing import image_resize
from city_sustainability.loading_data import loading_paths, image_and_label_arrays

2023-05-25 15:17:55.044445: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-25 15:17:55.285569: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-25 15:17:55.286578: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 15:17:56.361524: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Retrieve image_paths and label_paths
data_path_1 = '../../raw_data/OpenEarthMap_wo_xBD'
image_paths, label_paths = loading_paths(data_path_1)

In [3]:
# Check length of image_paths
print(len(image_paths))
# Check length of label_paths
print(len(label_paths))

2687
2687


In [4]:
# Import all image array as X and label array as y
X, y = image_and_label_arrays(image_paths, label_paths, sampling_ratio=1)

In [5]:
# Apply holdout
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [6]:
# Scale X and y
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [7]:
# Confirm shapes
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(1880, 256, 256, 3)
(807, 256, 256, 3)
(1880, 256, 256, 9)
(807, 256, 256, 9)
(1880, 256, 256, 3)
(807, 256, 256, 3)


In [23]:
class BaseModel:
    
    def fit(self, X, y):
        y_lb = np.argmax(y, axis=3)
        values, counts = np.unique(y_lb, return_counts=True)
        ind = np.argmax(counts)
        self.most_freq = values[ind]
        
    def predict(self, X):
        #y = self.most_freq_val * np.ones((X.shape))
        # Shape of our data
        output_shape = list(X.shape)
        output_shape[-1] = 9

        # Create the matrix with zeros
        y = np.zeros(output_shape)

        # Set ones everywhere for the most frequent class
        y[:, :, :, int(self.most_freq)] = 1

        # Print the matrix shape
        print(y.shape)
        
        return y

In [22]:
X_train.shape

(1880, 256, 256, 3)

In [24]:
bm = BaseModel()
bm.fit(X_train, y_train)

In [25]:
y_pred = bm.predict(X_test)

(807, 256, 256, 9)


In [27]:
iou_base_model = unet_model.compute_iou(y_test, y_pred)

2023-05-25 16:07:33.810106: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3807903744 exceeds 10% of free system memory.
2023-05-25 16:07:35.447198: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3807903744 exceeds 10% of free system memory.


In [29]:
iou_base_model.numpy()

0.12350264399016886

In [30]:
tf.metrics.accuracy(labels=y_test, predictions=y_pred)

AttributeError: module 'keras.api._v2.keras.metrics' has no attribute 'accuracy'